# Import libraries and data

In [55]:
import pandas as pd
import os
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

import pickle
with open("TX_data/variables.pickle", "rb") as f:
    me_2017,me_2018,me_2019,me_2021,me_2022,df_metric_2017,df_metric_2018,df_metric_2019,df_metric_2021,df_metric_2022,df_me_2017,df_me_2018,df_me_2019,df_me_2021,df_me_2022,metric_year,df3= pickle.load(f)

with open("TX_data/variables2.pickle", "rb") as f:
    df_me = pickle.load(f)

# Data Preprocess

## 1. abbreviations_explanation

In [ ]:
abb=pd.read_csv('TX_data/abbreviations_explanation.csv')
df_split = abb['meaning'].str.split('(', expand=True)
abb['meaning']=df_split[0]
abb['abbr'] = abb['abbr'].combine_first(df_split[1].str.replace(')', '') )
abb.to_csv('/Users/dai/Desktop/abbreviations.csv')

In [56]:
abb=pd.read_csv('TX_data/abbreviations.csv')

## 2. Aggregating TAPR_District Reports by Year

In [ ]:
folder_path = 'TX_data/TAPR_District/'

for year in [2017, 2018,2019,2021,2022]: 
    i=0
    merged_df = pd.DataFrame()
    
    subfolder_path = os.path.join(folder_path, str(year))
    
    for filename in os.listdir(subfolder_path):
         
        if filename.endswith('.csv')  or filename.endswith('.xls'):
            file_path = os.path.join(subfolder_path, filename)

            if filename.endswith('.csv'):
                df = pd.read_csv(file_path, index_col=0)
            elif filename.endswith('.xls'):
                df = pd.read_html(file_path)[0]
            if i==0:
                merged_df = pd.concat([merged_df, df])
            else:
                merged_df = pd.merge(merged_df, df,how='left',on='DISTRICT')
        i+=1
            
    merged_df.to_csv(folder_path+'TAPR_'+str(year)+'.csv')

## 3. Pre-process house price data

In [58]:
dh = pd.read_csv('TX_data/zip_code_market_tracker.tsv000', sep='\t')

# Select the desired area
dh=dh[dh['state']=='Texas']
dh['period_begin'] = pd.to_datetime(dh['period_begin'])
dh['period_end'] = pd.to_datetime(dh['period_end'])
dh=dh[dh['period_begin'].dt.year >= 2015]

dh['area']=dh['median_sale_price']/dh['median_ppsf']
dh['region']=dh['region'].str.replace('Zip Code: ','')
dh.rename(columns={'region':'zipcode'},inplace=True)
dh['zipcode'] = dh['zipcode'].astype(str)

dh=dh.sort_values('period_begin').reset_index(drop=True)
dh=dh.sort_values(['zipcode','period_begin','period_end','property_type_id'])

dh.head(3)

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,zipcode,city,state,...,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated,area
2460,2015-01-01,2015-03-31,90,zip code,2,32085,f,75001,NaN,Texas,...,NaN,NaN,NaN,0.266667,-0.233333,-0.019048,"Dallas, TX",19124,2023-03-12 14:49:18,1907.636540
574,2015-01-01,2015-03-31,90,zip code,2,32085,f,75001,NaN,Texas,...,NaN,NaN,NaN,0.666667,0.166667,0.541667,"Dallas, TX",19124,2023-03-12 14:49:18,1840.941176
1089,2015-01-01,2015-03-31,90,zip code,2,32085,f,75001,NaN,Texas,...,NaN,NaN,NaN,NaN,NaN,NaN,"Dallas, TX",19124,2023-03-12 14:49:18,1794.108260


In [59]:
# Choose related field
dh1=dh.loc[dh['property_type']=='All Residential',['zipcode', 'period_end','median_ppsf','area']]

# take last year's dec data as the next year data
## eg. 2022-12-01~2023-02-28 is considered as 2023
dh1=dh1[dh1['period_end'].isin(['2015-12-31','2016-12-31','2017-12-31','2018-12-31','2019-12-31','2020-12-31','2021-12-31','2022-12-31'])]
dh1['year'] = dh1['period_end'].dt.year
dh1['year']=dh1['year']+1

# calculate change percent
group = dh1.groupby('zipcode')
dh1['median_ppsf_prev'] = group['median_ppsf'].shift(1)
dh1['median_ppsf_change_percent'] = (dh1['median_ppsf'] - dh1['median_ppsf_prev']) / dh1['median_ppsf_prev'] *100
dh1=dh1.dropna(axis=0,how='any') #drop all rows that have any NaN values



dh1.head(3)

,zipcode,period_end,median_ppsf,area,year,median_ppsf_prev,median_ppsf_change_percent
57670,75001,2016-12-31,163.327769,1910.269165,2017,147.026432,11.087351
90049,75001,2017-12-31,179.076343,2091.750332,2018,163.327769,9.642313
121915,75001,2018-12-31,169.685669,2121.569859,2019,179.076343,-5.243950


## 4. Integration of District ID and Zipcode with House Data

Source: 
- https://schoolsdata2-tea-texas.opendata.arcgis.com/ > Archived Schools Data
- https://schoolsdata2-db440-tea-texas.opendata.arcgis.com/


In [180]:

# zipcode dataset
z=pd.read_csv('TX_data/School2020to2021.csv').sort_values('District_1')[[ 'District_1', 'District_N', 'Zip5']].drop_duplicates()
#z=z.applymap(lambda s: s.lower() if type(s) == str else s)
z.rename(columns={'Zip5':'zipcode','District_1':'DISTRICT','District_N':'DISTNAME'},inplace=True)
z['zipcode'] = z['zipcode'].astype(str)

display(z.head(3))

# merge with house price data
dt=pd.merge(dh1, z, on='zipcode', how='left')  #shape change: Because one zipcode may correspond to more distname
dt=dt[dt['DISTRICT'].notna()]
#dt['DISTRICT']=dt['DISTRICT'].astype('int')

display(dt.head(3))

# save data
dt.to_csv('TX_data/HousePrice/TX_houseprice.csv',index=False)

,DISTRICT,DISTNAME,zipcode
1351,1902,CAYUGA ISD,75832
2566,1903,ELKHART ISD,75839
2949,1904,FRANKSTON ISD,75763


,zipcode,period_end,median_ppsf,area,year,median_ppsf_prev,median_ppsf_change_percent,DISTRICT,DISTNAME
0,75001,2016-12-31,163.327769,1910.269165,2017,147.026432,11.087351,57905.0,DALLAS ISD
1,75001,2017-12-31,179.076343,2091.750332,2018,163.327769,9.642313,57905.0,DALLAS ISD
2,75001,2018-12-31,169.685669,2121.569859,2019,179.076343,-5.243950,57905.0,DALLAS ISD


## 5. Merge above data by year

In [181]:
dt_2020

,zipcode,period_end,median_ppsf,area,year,median_ppsf_prev,median_ppsf_change_percent,DISTRICT,DISTNAME
3,75001,2019-12-31,170.782974,1949.843077,2020,169.685669,0.646670,57905.0,DALLAS ISD
13,75002,2019-12-31,140.135794,2229.979877,2020,138.221154,1.385200,43901.0,ALLEN ISD
14,75002,2019-12-31,140.135794,2229.979877,2020,138.221154,1.385200,43919.0,LOVEJOY ISD
30,75006,2019-12-31,144.217788,1768.159146,2020,136.363636,5.759711,57828.0,WINFREE ACADEMY CHARTER SCHOOLS
31,75006,2019-12-31,144.217788,1768.159146,2020,136.363636,5.759711,57851.0,BRIDGEWAY PREPARATORY ACADEMY
...,...,...,...,...,...,...,...,...,...
11829,79935,2019-12-31,84.813084,1708.462810,2020,90.762404,-6.554829,71804.0,EL PASO ACADEMY
11830,79935,2019-12-31,84.813084,1708.462810,2020,90.762404,-6.554829,71905.0,YSLETA ISD
11846,79936,2019-12-31,94.482237,1450.008000,2020,87.094499,8.482439,71803.0,TRIUMPH PUBLIC HIGH SCHOOLS-EL PASO
11847,79936,2019-12-31,94.482237,1450.008000,2020,87.094499,8.482439,71905.0,YSLETA ISD


In [157]:
# creat new df by year
for year, group in dt.groupby('year'):
    globals()[f"dt_{year}"] = group.copy()
    globals()[f"dt_{year}"].to_csv('TX_data/HousePrice/house'+str(year)+'.csv')
    
# merge
for year in [2017, 2018,2019,2021,2022]: 
    globals()[f"TAPR_{year}"] = pd.read_csv('TX_data/TAPR_District/TAPR_'+str(year)+'.csv',na_values=['.'])
    globals()[f"TAPR_{year}"]['DISTRICT']=globals()[f"TAPR_{year}"]['DISTRICT'].str.replace("'",'').astype('float64')
    if 'Unnamed: 0' in globals()[f"TAPR_{year}"].columns:
        globals()[f"TAPR_{year}"]= globals()[f"TAPR_{year}"].drop(columns=['Unnamed: 0'])
    globals()[f"merged_{year}"] = pd.merge(globals()[f"dt_{year}"], globals()[f"TAPR_{year}"], on='DISTRICT', how='inner')

    # rename first DISTNAME_x
    col_names = list(globals()[f"merged_{year}"].columns)
    idx = col_names.index('DISTNAME_x')
    col_names[idx] = 'distname'
    globals()[f"merged_{year}"].columns = col_names
    
    # drop duplicate column 'DISTNAME_'
    drop_list=globals()[f"merged_{year}"].columns[globals()[f"merged_{year}"] .columns.str.startswith('DISTNAME')]
    globals()[f"merged_{year}"] = globals()[f"merged_{year}"].drop(columns=drop_list)
    
    # save data
    globals()[f"merged_{year}"].to_csv('TX_data/merged_'+str(year)+'.csv')

# Functions

## 1. Map District Names to their Corresponding IDs

In [4]:
# Input: a list of district name (default: all exist district name)
# Output: a dictionary, district_name: districtid.
def find_districtid(district_list=None):
    #did=pd.read_csv('TX_data/Districts2022to2023.csv')[['NAME','DISTRICT']].drop_duplicate()
    if district_list:
        did=pd.read_csv('TX_data/Districts2020to2021.csv')[['NAME','DISTRICT']]
        districtid = {}
        for d in district_list:
            id=did.loc[did['NAME']==d,'DISTRICT'].values[0]
            districtid[d] = id.replace('-','')
        #{'Dallas ISD': '057905', 'Fort Worth ISD': '220905'}
    else:
        did=pd.read_csv('TX_data/Districts2020to2021.csv')[['NAME','DISTRICT']]
        did['DISTRICT']=did['DISTRICT'].str.replace('-','')
        districtid=dict(zip(did['NAME'], did['DISTRICT']))
    return districtid

## 2. Retrieving Relevant IDs for a Given Field

In [111]:

def find_element(Data_field, Data_Level,Subcategory=None, year=None,detail='Y'):
    file_path = 'TX_data/datadict/district_processed/'
    if year is not None:
        csv_files = [os.path.join(file_path,'datadict_district_'+str(year)+'.csv' ),
                     os.path.join(file_path,'datadict_district_'+str(year-1)+'.csv' )]
    else:
        csv_files = [os.path.join(file_path, f) for f in os.listdir(file_path) if f.endswith(".csv") and f.startswith('datadict')]

    elements=[]
    for csv_file in csv_files:
        output = pd.read_csv(csv_file)
        
        # output = output.loc[output.iloc[:, 0] == Data_field, :]
        output = output.loc[(output.iloc[:, 0].str.startswith(Data_field)) & (~output.iloc[:, 0].isna()), :]
        
        if Subcategory is not None:
            #output=output.loc[ output.iloc[:,1]==Subcategory ,:]
            output=output.loc[ output.iloc[:,1].str.startswith(Subcategory) ,:]

        if year is not None:
            output=output.loc[ output.iloc[:,2]==str(year),: ]

        element = output.loc[:, Data_Level]
        if not element.empty:
            elements.append(element.values[0])
    
    print('The relevant element ids are: ',elements,'\n')
    
    if detail=='Y':
        if type(elements)=='str':
            elements=[elements]
        for enum in elements:
            try:
                position = df_me[df_me['elements'].apply(lambda x: enum in x)].index.tolist()[0]
            except:
                print('please make sure ', enum,' not belong to State and Region ## column')
                
            e_years=df_me.loc[position,'occurrence_year']
            e_metric=df_me.loc[position,'metric']
            meaning=df_me.loc[position,'meaning']
            print(enum,' belongs to metric: ',e_metric) 
            print('Occurrence year: ', e_years)
            print("Meaning: ",meaning)  
            print('\n')      
                
    return elements

## 3. District Dataframe Retrieval Function: Explaining its Elements and Usage

In [36]:
def select_columns(df, columns):
    existing_columns = set(df.columns)
    selected_columns = [col for col in columns if col in existing_columns]
    return df[selected_columns]

def mapping5_list(enum_list):
    path_list=[]
    ae=[] 
    if type(enum_list)=='str':
        enum_list=[enum_list]
    for enum in enum_list:
        try:
            position = df_me[df_me['elements'].apply(lambda x: enum in x)].index.tolist()[0]
        except:
            print('please make sure ', enum,' not belong to State and Region ## column')
            continue
            
        e_years=df_me.loc[position,'occurrence_year']
        e_metric=df_me.loc[position,'metric']
        
        # column name
        for ind,i in enumerate(e_years):
            ae.append(enum[:-3] + str(i)[2:4] + enum[-1])
            ae.append(enum[:-3] + str(i-1)[2:4] + enum[-1])
            ae.append(enum[:-3] + str(i-2)[2:4] + enum[-1])
            path_list.append(os.path.join('TX_data','TAPR_District',str(i), e_metric))
            
    ae=list(set(ae))
    ae.sort()
    ae.insert(0,'DISTRICT')
    path_list=list(set(path_list))
    path_list.sort()
        
    for ind,path in enumerate(path_list):
        if int(path.split('/')[2])<=2019:
            df = pd.read_html(path+'.xls')[0]
        else:
            df = pd.read_csv(path+'.csv')
        dc= select_columns(df, ae)
        
        if ind==0:
            output=dc
        else:
            output=pd.merge(output,dc,on='DISTRICT',how='outer')
    
    # Deal with duplicate columns. Save the one occur in later year.
    output = output[output.columns[~output.columns.str.endswith('_x')]]
    output=output.rename(columns={col: col.rstrip('_y') for col in output.columns if col.endswith('_y')})
    output.sort_index(axis=1, ascending=False, inplace=True)
    
    # Sort columns by its name
    cols = output.columns.tolist()
    cols.sort()
    cols.remove('DISTRICT')
    cols.insert(0, 'DISTRICT')
    output = output[cols]
    output.iloc[:,0]=output.iloc[:,0].str.strip("'")
    output = output.set_index(output.columns[0])
    return output

## Running Example

In [39]:
# 1. give spefic district name, return its id
districtid0=find_districtid(district_list=None)
print(districtid0)

district_list=['Dallas ISD','Fort Worth ISD']
districtid=find_districtid(district_list)
print(districtid)

{'Memphis ISD': '096904', 'Patton Springs ISD': '063906', 'Spur ISD': '063903', 'Sherman ISD': '091906', 'S and S Cons ISD': '091914', 'Pottsboro ISD': '091913', 'Waxahachie ISD': '070912', 'Palmer ISD': '070910', 'Ennis ISD': '070903', 'Denison ISD': '091903', 'Bells ISD': '091901', 'Whitesboro ISD': '091909', 'Collinsville ISD': '091902', 'Evadale ISD': '121906', 'Buna ISD': '121903', 'Santa Gertrudis ISD': '137904', 'Riviera ISD': '137903', 'Ricardo ISD': '137902', 'Kingsville ISD': '137901', 'Apple Springs ISD': '228905', 'Trinity ISD': '228903', 'Kennard ISD': '113906', 'Jefferson ISD': '155901', 'Avinger ISD': '034902', 'Pecos-Barstow-Toyah ISD': '195901', 'Balmorhea ISD': '195902', 'Terrell County ISD': '222901', 'Dilley ISD': '082902', 'Springlake-Earth ISD': '140907', 'Tahoka ISD': '153904', "O'Donnell ISD": '153903', 'Dawson ISD (Dawson)': '058902', 'Silverton ISD': '023902', 'Nazareth ISD': '035903', 'Hart ISD': '035902', 'Dimmitt ISD': '035901', 'Stamford ISD': '127906', 'P

In [112]:
# 2.1 give wanted field
test=pd.read_csv('TX_data/5years_trends_try.csv')
Data_field=test['Data field'][0]
Subcategory	= test['Subcategory'][0]
Data_Level=test['Data Level'][0]
print('The field we want is: '+Data_field,Subcategory,Data_Level,sep='; ')

# 2.2 return element id
enum_list=find_element(Data_field,Data_Level, Subcategory, year=None,detail='Y')  

The field we want is: All Grades ELA/Reading; At Meets Grade Level or Above; District
The relevant element ids are:  ['DDA00AR01218R', 'DDA00AR01219R', 'DDA00AR01221R', 'DDA00AR01222R'] 

DDA00AR01218R  belongs to metric:  DBIL1
Occurrence year:  [2017, 2018, 2019, 2021, 2022]
Meaning:  BE/ESL: Approaches Grade Level, Meets Grade Level, and Masters Grade Level


DDA00AR01219R  belongs to metric:  DBIL1
Occurrence year:  [2017, 2018, 2019, 2021, 2022]
Meaning:  BE/ESL: Approaches Grade Level, Meets Grade Level, and Masters Grade Level


DDA00AR01221R  belongs to metric:  DBIL1
Occurrence year:  [2017, 2018, 2019, 2021, 2022]
Meaning:  BE/ESL: Approaches Grade Level, Meets Grade Level, and Masters Grade Level


DDA00AR01222R  belongs to metric:  DBIL1
Occurrence year:  [2017, 2018, 2019, 2021, 2022]
Meaning:  BE/ESL: Approaches Grade Level, Meets Grade Level, and Masters Grade Level




In [37]:
# 3. return the result dataframe. 
if enum_list != None:
    dt=mapping5_list(enum_list)

dt=dt.loc[list(districtid.values()), :]  #bc: the number of `districtid`` less than actual `district``
dt = dt.replace('.', np.nan).astype(float)
new_rows = {v: k for k, v in districtid.items()}  
dt = dt.rename(index=new_rows)
display(dt)

,DDA00AR01218R,DDA00AR01219R,DDA00AR01221R,DDA00AR01222R
DISTRICT,,,,
Dallas ISD,38.0,41.0,37.0,43.0
Fort Worth ISD,35.0,35.0,30.0,38.0


# General EDA

In [43]:
def factor_name(element):
    year=element[-3:-1]
    file_path = 'TX_data/datadict/district_processed/'
    csv_files = [os.path.join(file_path, f) for f in os.listdir(file_path) if f.endswith('.csv') and str(year) in f][0]
    df2=pd.read_csv(csv_files)
    mask = df2.where(df2 == element)
    stacked = mask.stack()
    info=', '.join([df2.iloc[stacked.index.get_level_values(0), 0].values[0],df2.iloc[stacked.index.get_level_values(0), 1].values[0], stacked.index.get_level_values(1).values[0], str(year)])
    info_dict={element:info}
    return info_dict
## eg. element = 'DDA00AR01218R'

def analysis(df,top=None):
    
    df_mean = df.mean(axis=1) 
    if top!=None:
        df_sorted = df.mean().sort_values(ascending=False)
        cols_top = df_sorted.index[:top]
        df = df[cols_top]
    
    df_mean_select = df.mean(axis=1)
    
    fig = go.Figure()
    
    # Add scatter plot for each column
    fig = px.scatter(df, x=df.index, y=df.columns, title='Scatter Plot')
    fig.add_trace(go.Scatter(x=df.index, y=df_mean_select, name='Select average', mode='markers', line=dict(color='black', width=4)))
    fig.add_trace(go.Scatter(x=df.index, y=df_mean, name='Overall average', mode='markers', line=dict(color='grey', width=4)))
    fig.show()
    
    for col in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df[col], name=col))
    fig.add_trace(go.Scatter(x=df.index, y=df_mean_select, name='Select average', line=dict(color='black', width=3)))
    fig.add_trace(go.Scatter(x=df.index, y=df_mean, name='Overall average', line=dict(color='grey', width=3)))
    fig.update_layout(title='Line Plot', xaxis_title='X Axis', yaxis_title='Y Axis')
    fig.show()

    # Add box plot for each column
    fig = px.box(df, y=df.columns, title='Box Plot')
    fig.add_trace(go.Box(y=df_mean_select, name='Select average', line=dict(color='black', width=3)))
    fig.add_trace(go.Box(y=df_mean, name='Overall average', line=dict(color='grey', width=3)))
    fig.show()

    # Add pct change plot for each column
    df_pct_change = df.pct_change()
    df_pct_change_long = df_pct_change.melt(value_name='pct_change', var_name='column')
    fig = px.line(df_pct_change_long, x='column', y='pct_change', title='Pct Change Trend Analysis')
    fig.show()

In [45]:
df=dt.T.copy()

info_dict = {}
for element in df.index:
    info_dict.update(factor_name(element))

for key, value in info_dict.items():
    print(f"{key}: {value}")
    
analysis(df,top=30)

DDA00AR01218R: All Grades ELA/Reading, At Meets Grade Level or Above, District, 18
DDA00AR01219R: All Grades ELA/Reading, At Meets Grade Level or Above, District, 19
DDA00AR01221R: All Grades ELA/Reading, At Meets Grade Level or Above, District, 21
DDA00AR01222R: All Grades ELA/Reading, At Meets Grade Level or Above, District, 22


# Feature engineering for house data
see another notebook

# Pipeline

In [ ]:
mapping5_list(['DF0GZ21B']) # why error

#dh[dh.isna().any(axis=1)] # there is NaN value.
#dh=dh.loc[:,['period_begin', 'period_end','region','median_ppsf', 'median_sale_price','median_list_ppsf','median_list_price','property_type','property_type_id','parent_metro_region', 'parent_metro_region_metro_code']]

In [101]:
find_element(Data_field='Graduates and TxCHSE', Data_Level=None,Subcategory=None, year=None,detail='Y')

KeyError: None

In [110]:
Data_Level='District'
year=None
detail='Y'
file_path = 'TX_data/datadict/district_processed/'
csv_files = [os.path.join(file_path, f) for f in os.listdir(file_path) if f.endswith(".csv") and f.startswith('datadict')]
elements=[]


# 从这里开始改原来的函数

Data_field=None
#Data_field='4-Year Longitudinal Rate (Gr 9-12)'
#Data_field='Graduates and TxCHSE'

#Subcategory='Graduates and GED'
Subcategory='Graduates'


output = pd.read_csv(csv_files[1])
if Data_field is not None:
    output = output.loc[(output.iloc[:, 0].str.startswith(Data_field)) & (~output.iloc[:, 0].isna()), :]

if Subcategory is not None:
    #output=output.loc[ output.iloc[:,1]==Subcategory ,:]
    output=output.loc[ output.iloc[:,1].str.startswith(Subcategory) ,:]

output
# 这个有问题。之前合并的时候存在一些。目前只有2017按我想要的方式运行了

,Data field,Subcategory,Unnamed: 1,State,Region ##,District,African American,Hispanic,White,American Indian,Asian,Pacific Islander,Two or More Races,Special Ed,Econ Disadv,EL,Unnamed: 15
273,Class of 2017,Graduates and TxCHSE,SA3C417R,RA3C417R,DA3C4X17R,DB3C4X17R,DH3C4X17R,DW3C4X17R,DI3C4X17R,D33C4X17R,D43C4X17R,D23C4X17R,DS3C4X17R,DE3C4X17R,DL3C4X17R,NaN,NaN
280,Class of 2016,Graduates and TxCHSE,SA3C416R,RA3C416R,DA3C4X16R,DB3C4X16R,DH3C4X16R,DW3C4X16R,DI3C4X16R,D33C4X16R,D43C4X16R,D23C4X16R,DS3C4X16R,DE3C4X16R,DL3C4X16R,NaN,NaN
288,Class of 2016,"Graduates and TxCHSE Grads, TxCHSE, & Cont",SA3C516R SA2C516R,RA3C516R RA2C516R,DA3C5X16R DA2C5X16R,DB3C5X16R DB2C5X16R,DH3C5X16R DH2C5X16R,DW3C5X16R DW2C5X16R,DI3C5X16R DI2C5X16R,D33C5X16R D32C5X16R,D43C5X16R D42C5X16R,D23C5X16R D22C5X16R,DS3C5X16R DS2C5X16R,DE3C5X16R DE2C5X16R,DL3C5X16R DL2C5X16R,NaN,NaN
294,Class of 2015,Graduates and TxCHSE,SA3C515R,RA3C515R,DA3C5X15R,DB3C5X15R,DH3C5X15R,DW3C5X15R,DI3C5X15R,D33C5X15R,D43C5X15R,D23C5X15R,DS3C5X15R,DE3C5X15R,DL3C5X15R,NaN,NaN
304,Class of 2015,"Graduates and TxCHSE Grads, TxCHSE, & Cont",SA3C615R SA2C615R,RA3C615R RA2C615R,DA3C6X15R DA2C6X15R,DB3C6X15R DB2C6X15R,DH3C6X15R DH2C6X15R,DW3C6X15R DW2C6X15R,DI3C6X15R DI2C6X15R,D33C6X15R D32C6X15R,D43C6X15R D42C6X15R,D23C6X15R D22C6X15R,DS3C6X15R DS2C6X15R,DE3C6X15R DE2C6X15R,DL3C6X15R DL2C6X15R,NaN,NaN
310,Class of 2014,Graduates and TxCHSE,SA3C614R,RA3C614R,DA3C6X14R,DB3C6X14R,DH3C6X14R,DW3C6X14R,DI3C6X14R,D33C6X14R,D43C6X14R,D23C6X14R,DS3C6X14R,DE3C6X14R,DL3C6X14R,NaN,NaN
343,Graduates (2016-17 Annual Graduates),Graduates (2016-17 Annual Graduates),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
464,Graduates Enrolled in Texas Institution of Hig...,Graduates Enrolled in Texas Institution of Hig...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
467,Graduates in TX IHE Completing One Year Withou...,Graduates in TX IHE Completing One Year Withou...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
